In [ ]:
get_ipython().magic(u'matplotlib inline')
import matplotlib.pyplot as plt
from urllib2 import Request, urlopen, URLError
import numpy as np
from skimage.util import view_as_windows as vaw
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences
import tensorflow as tf
from scipy.stats import mode
from numpy.linalg import norm

In [ ]:
kw = 'homeobox'  # the keyword to search uniprot for
test_len = 100  # the length to cut each sequence up into
sequences = {}

In [ ]:
# homeobox
sequences['homeobox'] = ['r', 'r', 'r', 'k', 'r', 't', 'a', 'y',
                         't', 'r', 'y', 'q','l', 'l', 'e', 'l', 'e',
                         'k', 'e', 'f', 'h', 'f', 'n', 'r', 'y', 'l',
                         't', 'r', 'r', 'r', 'r', 'i', 'e', 'l', 'a',
                         'h', 's', 'l', 'n', 'l', 't', 'e','r', 'h',
                         'i', 'k', 'i', 'w', 'f', 'q', 'n', 'r', 'r',
                         'm', 'k', 'w', 'k', 'k', 'e', 'n']

In [ ]:
# kw-0440 - LIM
sequences['KW-0440'] = ['c', 'x', 'x', 'x', 'x', 'c', 'x', 'x',
                        'x', 'x', 'x', 'x', 'x', 'x', 'x','x', 'x',
                        'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'w', 'h',
                        'x', 'x', 'x', 'x', 'c', 'f', 'x', 'c', 'x', 'x',
                        'x', 'x', 'c', 'x', 'x', 'x', 'x', 'x', 'x', 'x',
                        'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x',
                        'x', 'x', 'x', 'c', 'x', 'x', 'x', 'x', 'c']

In [ ]:
# kw-0501
consensus = 'MQAEILLTLKLQQKLFADPRRISLLKHIALSGSISQGAKDAGISYKSAWDAINEMNQLSEHILVERATGGKGGGGAVLTRYGQRLIQLYDLLAQIQQKAFDVLSDDDALPLNSLLAAISRFSLQTSARNQWFGTITARDHDDVQQHVDVLLADGKTRLKVAITAQSGARLGLDEGKEVLILLKAPWVGITQDEAVAQNADNQLPGIISHIERGAEQCEVLMALPDGQTLCATVPVNEATSLQQGQNVTAYFNADSVIIATLC'
sequences['KW-0501'] = map(lambda x:x.lower(), list(consensus))

In [ ]:
def letter2num(c, length):
    '''Assigns each letter in the amino acid code c
       a unique integer and chops the sequence into
       shorter sequences.'''
          
    try:
        len(c[0])
        X = np.zeros([0, length])
        
        for i in xrange(len(c) - 1):
            if len(c[i]) >= length:
                x = []
                
                for j in xrange(len(c[i])):
                    x.append(max(ord(c[i][j])-97, 0))
                    
                X = np.concatenate((X, vaw(np.asarray(x), (length,))), 0)
                
    except TypeError:
        X = np.zeros([len(c)])
        for i in xrange(len(c)):
            X[i] = (ord(c[i])-97)
        
        if length is not None:
            X = vaw(X, (length, ))
            
    return X

In [ ]:
consensus = sequences[kw]
c_length = len(consensus)  # length of the respective consensus sequence
c = letter2num(consensus, len(consensus))

In [ ]:
def get_uniprot_data(query, limit):
    '''Goes to the uniprot website and searches for 
       data with the keyword query. Returns the data 
       found up to limit elements.'''
    
    url1 = 'http://www.uniprot.org/uniprot/?query'
    url2 = '&columns=sequence&format=tab&limit='+str(limit)
    query_complete = url1 + query + url2
    request = Request(query_complete)
    response = urlopen(request)
    data = response.read()
    data = data.split('\n')
    data = data[1:]
    
    return map(lambda x:x.lower(),data)

In [ ]:
def LSTM(net, d_prob):
    '''The LSTM model used for training'''
    
    net = tflearn.embedding(net, input_dim=25, output_dim=25)
    net = tflearn.lstm(net, 300, dropout=d_prob, dynamic=False, return_seq=True)
    # net = tflearn.lstm(net, 140, dropout=0.5, dynamic=True, return_seq=True)
    net = tflearn.lstm(net, 300)
    net = tflearn.fully_connected(net, 150, activation='tanh')
    net = tflearn.fully_connected(net, 2, activation='softmax')
    return net

In [ ]:
def num2letter(seq):
    '''Takes in a seq of amino acids composed of numbers 
       and transforms them into the respective letters.'''
    
    letters = []
    
    for i in xrange(len(seq)):
        letters.append(chr(int(seq[i]+97)))
        
    return letters

In [ ]:
# loop through the training procedure with different sub-sequence
# lengths
for i in xrange(60, 151, 5):
    
    # reset the tensorflow graph each time we change the 
    # sequence size
    tf.reset_default_graph()
    
    # get the data from the uniprot webpage
    # in this case, were pulling 1300 proteins 
    # in each category
    x1 = get_uniprot_data('=' + kw, 2000)
    x0 = get_uniprot_data('=NOT+' + kw, 2000)
    
    # Transform each letter in the amino acid sequence
    # into a unique integer
    x1=letter2num(x1, i)
    x0=letter2num(x0, i)
    data_length = x0.shape[0]

    # make the number of examples equal for each group
    # by shortening x1, which is the one with more data
    x1=x1[:x0.shape[0], :]
    
    # put all examples from both groups into one variable X
    X = np.concatenate((x1, x0), 0)
    
    # Create labels and make them into one-hot vectors with
    # two dimensions
    Y = np.zeros([x1.shape[0]+x0.shape[0], ])
    Y[:x1.shape[0]] = 1.0
    Y = to_categorical(Y, 2)

    # send the input placeholder to the lstm 
    net = tflearn.input_data([None, i])
    net = LSTM(net, 0.5) # get the output of the network
    
    # choose the backprop algorithm, learning rate, and
    # objective function
    net = tflearn.regression(net, optimizer='adam',
                             learning_rate=0.00001,
                             loss='categorical_crossentropy')
    
    # instantiate the model
    model = tflearn.DNN(net, tensorboard_verbose=1)
    
    # call the training method with its parameters
    model.fit(X, Y, 
          n_epoch=10, 
          validation_set=0.25, 
          shuffle=True, 
          show_metric=True, 
          batch_size=70,
          snapshot_step=10000,
          run_id='ProteinNet_' + str(i) + '_' + str(data_length) + '_LSTM_' + str(np.random.randint(1, 10**4)))
    
    model.save('lstm' + str(i) + kw)

In [ ]:
# send the input placeholder to the lstm 
net = tflearn.input_data([None, test_len])
net = LSTM(net, 1.0) # get the output of the network

# instantiate the model
model = tflearn.DNN(net, tensorboard_verbose=1)

# load the saved model you want to test
model.load('lstm' + str(test_len), weights_only=True)
print(model)

In [ ]:
num_test = 2000 # number of proteins to test
X1 = get_uniprot_data('=' + kw, num_test)
best = np.zeros([0, test_len])

In [ ]:
for i in xrange(num_test):
    x1 = X1[i]
    
    if len(x1) > test_len:
        x1 = letter2num(x1, test_len)
        x1out = np.asarray(model.predict(x1))
        
        if np.amin(x1out[:, 1])-np.amax(x1out[:, 0]) > 0.5:
            print(np.mean(x1out[:, 1])-np.mean(x1out[:, 0]))
            best_ind = np.argmax(x1out[:, 1])

            # plot the activation of each output node for each seq. fragment
            fig = plt.figure()
            a0 = fig.add_subplot(111)
            a0.set_ylabel('activation of output layer')
            a0.set_xlabel('sequence fragment')
            a0.plot(x1out[:, 0], label = 'node 0')
            a0.plot(x1out[:, 1], label='node 1')
            a0.scatter(best_ind, 0)
            plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
            plt.tight_layout()

In [ ]:
# check the consensus and pad it to same size as network input
if c.shape != (test_len,):
    c = c[0, :]
    c = np.pad(c, ((test_len-c_length)/2, (test_len-c_length)/2),
               'constant', constant_values=-1.)
    print(c.shape)

In [ ]:
# run the consensus sequence back through the model to
# see the activity of the output layer
output = model.predict(c[None, :])
print(np.amax(output))